Imagine we sell icecreams in the local park. We want to know how many ice-creams we are going to sell each day because we need to buy enough icecream and cones for everyone without having too much waste left over at the end of the day. We know how many icecreams we sold on different days in the past - can we use this data to predict how many we might sell in the future.

First we need some libraries for python.
- Pandas allows us to use tables for our data.
- Scikit-learn (sklearn) is one of the main python libraries for many different machine learning models.
- Matplotlib allows us to make graphs of our data.

In [ ]:
import pandas
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


In order to see how well our model performs, we will want to train the model on some of our data and then test it on data our model has not yet seen. In this case, the train and test data have been split into two different files already.


In [ ]:
icream_sales_train_data = pandas.read_csv(r"TableSalesPredictCSV.csv")

In [ ]:
# The .head command allows us to see the first few rows of our data in the table.
icream_sales_train_data.head()

Data Analysis

The first thing we tend to do with any new data project is use graphs and some simple statistics to look at whether we can see a relationship between the data we have (features) and what we want to predict (target).

For example, if there is a funfair on, do we sell more icecream?

In [ ]:
plt.bar(icream_sales_train_data["Local funfair"], icream_sales_train_data["Icecreams sold"])
plt.show()

Yes - it looks like when there is a funfair on we sell more icecream.

This is possibly unsurprising - the local funfair probably means more people coming to the park so more people to buy icecream.

How about the temperature?

In [ ]:
plt.scatter(icream_sales_train_data["Temperature"], icream_sales_train_data["Icecreams sold"])
plt.xlabel("Temperature")
plt.ylabel("Icecreams sold")

plt.show()

So again, we can see a pattern - if it's hotter, we tend to sell more icecream. Maybe this will be useful in our model as well. How about day of the week? We could do this using another bar graph but just comparing the averages doesn't always tell us enough. 
We can look at the spread of our values using something called a boxplot. A boxplot, takes all our values and lines them up in increasing order. Then we see a line (in this case in orange) where the middle value sits and a box around the middle half of our values. The lowest value and highest value are also shown.

In [ ]:
# To draw a boxplot, we need to provide a list of data points for each day of the week. We can pull these out of
# the table by selecting all the rows where 'Day of the week' is the day we want and then selecting 
# the 'Icecreams sold' column
icecreams_monday = icream_sales_train_data.loc[icream_sales_train_data["Day of the week"] == "Monday"]["Icecreams sold"]
icecreams_tuesday = icream_sales_train_data.loc[icream_sales_train_data["Day of the week"] == "Tuesday"]["Icecreams sold"]
icecreams_wednesday = icream_sales_train_data.loc[icream_sales_train_data["Day of the week"] == "Wednesday"]["Icecreams sold"]
icecreams_thursday = icream_sales_train_data.loc[icream_sales_train_data["Day of the week"] == "Thursday"]["Icecreams sold"]
icecreams_friday = icream_sales_train_data.loc[icream_sales_train_data["Day of the week"] == "Friday"]["Icecreams sold"]
icecreams_saturday = icream_sales_train_data.loc[icream_sales_train_data["Day of the week"] == "Saturday"]["Icecreams sold"]
icecreams_sunday = icream_sales_train_data.loc[icream_sales_train_data["Day of the week"] == "Sunday"]["Icecreams sold"]

In [ ]:
plt.boxplot([icecreams_monday, icecreams_tuesday, icecreams_wednesday, icecreams_thursday, icecreams_friday, icecreams_saturday, icecreams_sunday], labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plt.show()

So it looks like, in general, we sell more icecream at the weekend. Although we do also seem to sell lots of icecream on a Thursday as well...

We also have some data on the price of catfood in the local supermarket? Do we think this makes a difference to how many icecreams we will sell?

In [ ]:
plt.scatter(icream_sales_train_data["Price of catfood at local supermarket"], icream_sales_train_data["Icecreams sold"])
plt.xlabel("Temperature")
plt.ylabel("Icecreams sold")

plt.show()

Not really any relationship here. And we probably wouldn't expect one. Why would the price of catfood in the local supermarket affect how many icecreams we sell?

So how abour price of icecream in the local supermarket? We might think if icecream is really cheap in the local supermarket then people might choose to buy it there instead of from our icecream stand.

In [ ]:
plt.scatter(icream_sales_train_data["Price of icecream at local supermarket"], icream_sales_train_data["Icecreams sold"])
plt.xlabel("Temperature")
plt.ylabel("Icecreams sold")

plt.show()

We don't see a strong relationship here either. Maybe we will leave this out for now as well.

Data Preparation

Now that we know which features we want, we need to get them into the right format for our modeling. Models understand numbers but, for example, day of the week is categorical (one of several categories). We can convert categories into numbers by using one hot encoding. However, in this case, we may also want to just split the days into whether the day is a weekend or not.

To do this, we create another column in our table and map the 'Day of the week' column to 1 if the day is a Saturday or Sunday or 0 otherwise.

In [ ]:
icream_sales_train_data["isWeekend"] = icream_sales_train_data["Day of the week"].map(lambda x: 1 if x == "Saturday" or x == "Sunday" else 0)


We can do the same with 'Will it rain?'


In [ ]:
icream_sales_train_data["rains"] = icream_sales_train_data["Will it rain?"].map(lambda x: 1 if x == "Yes" else 0)


In [ ]:
icream_sales_train_data.head()

Modeling the data

Now we can try and build a model that predicts the number of icreams sold based on our features of 'Temperature', 'isWeekend' and 'rains'.

In [ ]:

X = icream_sales_train_data[["Temperature", "isWeekend", "rains"]]

Y = icream_sales_train_data["Icecreams sold"]

reg = LinearRegression().fit(X, Y)

How well does this model predict our sales?

In [ ]:
reg.score(X, Y)

This is a measure of our accuracy, with 0 being not accurate at all and 1 being completely accurate. So getting nearly 0.9 is not bad. But, this is using the data that we trained the model on - we expect it to be accurate. How about the data we 'held out' as test data?

In [ ]:
test_data_icecream_sales = pandas.read_csv(r"TableSalesPredictTestDataCSV.csv")

In [ ]:
test_data_icecream_sales

First, we need to map our features to numbers like we did with the training data.

In [ ]:
test_data_icecream_sales["isWeekend"] = test_data_icecream_sales["Day of the week"].map(lambda x: 1 if x == "Saturday" or x == "Sunday" else 0)
test_data_icecream_sales["rains"] = test_data_icecream_sales["Will it rain?"].map(lambda x: 1 if x == "Yes" else 0)


Now we can predict how many icecreams we would expect to sell for each row of our test data using our model

In [ ]:
reg.predict(test_data_icecream_sales[["Temperature", "isWeekend", "rains"]])

Let's see how accurate that is

In [ ]:
reg.score(test_data_icecream_sales[["Temperature", "isWeekend", "rains"]], test_data_icecream_sales["Icecreams sold"])

But wait, we forgot to include the funfair... This can be included in the same way as whether it was raining. Does this make our model better or worse?